In [10]:
import numpy as np
import pandas as pd
from math import cos, asin, sqrt
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
df = pd.read_csv(r"C:\Users\ADMIN\Desktop\Work\route\in.csv")
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Delhi,28.6100,77.2300,India,IN,Delhi,admin,32226000,16753235
1,Mumbai,19.0761,72.8775,India,IN,Mahārāshtra,admin,24973000,12478447
2,Kolkāta,22.5675,88.3700,India,IN,West Bengal,admin,18502000,4496694
3,Bangalore,12.9789,77.5917,India,IN,Karnātaka,admin,15386000,8443675
4,Chennai,13.0825,80.2750,India,IN,Tamil Nādu,admin,12395000,6727000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city               162 non-null    object 
 1   lat                162 non-null    float64
 2   lng                162 non-null    float64
 3   country            162 non-null    object 
 4   iso2               162 non-null    object 
 5   admin_name         162 non-null    object 
 6   capital            19 non-null     object 
 7   population         162 non-null    int64  
 8   population_proper  162 non-null    int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 11.5+ KB


In [4]:
df = df[df.city!= 'Kolkata']
cities_lat_dict = df[['city', 'lat']].set_index('city')['lat'].to_dict()
cities_lng_dict = df[['city', 'lng']].set_index('city')['lng'].to_dict()

In [5]:
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Delhi,28.6100,77.2300,India,IN,Delhi,admin,32226000,16753235
1,Mumbai,19.0761,72.8775,India,IN,Mahārāshtra,admin,24973000,12478447
2,Kolkāta,22.5675,88.3700,India,IN,West Bengal,admin,18502000,4496694
3,Bangalore,12.9789,77.5917,India,IN,Karnātaka,admin,15386000,8443675
4,Chennai,13.0825,80.2750,India,IN,Tamil Nādu,admin,12395000,6727000


In [7]:
def DistanceMatrix(x1, y1, x2, y2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((x2 - x1) * p)/2 + cos(x1 * p) * cos(x2 * p) * (1 - cos((y2 - y1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [8]:
def create_data_model():
  nodes = [0] ## initializing with the depot node as 0
  lat = {}
  lng = {}
  original_id_dict = {}

  depot_node = [0]
  original_id_dict[0] = 'Kolkata'
  lat[0] = 22.5411
  lng[0] = 88.3378


  cust_loc = 1
  for city in cities_lat_dict.keys():
    lat[cust_loc] = cities_lat_dict[city]
    lng[cust_loc] = cities_lng_dict[city]
    original_id_dict[cust_loc] = city
    nodes.append(cust_loc)
    cust_loc +=1

  location = {} 
  distance = {}
  for from_node in nodes:
    location[from_node] = (lat[from_node], lng[from_node])
    distance[from_node] = {}
    for to_node in nodes:
        distance[from_node][to_node] = DistanceMatrix(lat[from_node], lng[from_node],
                                          lat[to_node], lng[to_node])
        
  data = {}
  data["locations"] = location
  data["num_locations"] = len(data["locations"])
  data["num_vehicles"] = 1
  data["depot"] = 0
  data["distance_matrix"] = distance
  data["original_id_dict"] = original_id_dict
  return data

In [9]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            
            plan_output += ' {} -> '.format(data["original_id_dict"][manager.IndexToNode(index)])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(data["original_id_dict"][manager.IndexToNode(index)])
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        if route_distance>0:
            print(plan_output)
            total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))

In [11]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Improve the initial solution by a meta-heuristic algorithm

    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    # search_parameters.time_limit.FromSeconds(10)
    search_parameters.time_limit.seconds = 2
    search_parameters.log_search = True

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [12]:
if __name__ == '__main__':
    main()

Objective: 12159
Route for vehicle 0:
 Kolkata ->  Taliganja ->  Nagtala ->  Zeyādah Kot ->  Santoshpur ->  Baj Baj ->  Bāuria ->  Balasore ->  Kendrāparha ->  Puri ->  Jamshedpur ->  Harilādih ->  Dhanbād ->  Sijua ->  Rānchi ->  Bilāspur ->  Raipur ->  Vishākhapatnam ->  Ambāpuram ->  Vijayavāda ->  Tiruvottiyūr ->  Manali ->  Chinnasekkadu ->  Madhavaram ->  Ambattūr ->  Nerkunram ->  Chennai ->  Madipakkam ->  Pallāvaram ->  Nanmangalam ->  Bommayapālaiyam ->  Oulgaret ->  Puducherry ->  Pāppākurichchi ->  Gundūr ->  Madurai ->  Bendravādi ->  Sondekoppa ->  Kitanelli ->  Kadabgeri ->  Kodagihalli ->  Herohalli ->  Cāchohalli ->  Dāsarhalli ->  Jālhalli ->  Bayandhalli ->  Bangalore ->  Doddajīvanhalli ->  Kammanhalli ->  Byatarayanpur ->  Abbigeri ->  Chik Bānavar ->  Mādavar ->  Mādnāikanhalli ->  Adakimāranhalli ->  Arshakunti ->  Vājarhalli ->  Hesarghatta ->  Kasgatpur ->  Yelahanka ->  Hunasamaranhalli ->  Sonnappanhalli ->  Kādiganahalli ->  Huttanhalli ->  Bāgalūr ->  Maila